In [33]:
# %pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [35]:
from PyPDF2 import PdfReader

reader = PdfReader("docs/regulamento-dos-cursos-de-graduacao-da-UFRN-2024.pdf")
len(reader.pages)

77

In [58]:
raw_text = ""

for i in range(len(reader.pages)):
    page = reader.pages[i]
    text = page.extract_text()
    text.strip()

    raw_text += text

print(len(raw_text))

173365


In [96]:
titulo_starts = [pos for pos, _ in enumerate(raw_text) if raw_text.startswith('TÍTULO', pos)]
secoes_starts = [pos for pos, _ in enumerate(raw_text) if raw_text.startswith('Seção', pos)]
artigo_starts = [pos for pos, _ in enumerate(raw_text) if raw_text.startswith('Art.', pos)]
# paragrafo_starts = [pos for pos, _ in enumerate(raw_text) if raw_text.startswith('Parágrafo único.', pos)]
# p_starts = [pos for pos, _ in enumerate(raw_text) if raw_text.startswith('§', pos)]

In [111]:
section_starts = titulo_starts + secoes_starts + artigo_starts
section_starts.sort()

In [115]:
splits = []

for i in range(len(section_starts) - 1):
    section = raw_text[section_starts[i]:section_starts[i+1]]
    splits.append(section)

splits[:15]

['Art. 1º  Atualizar o Regulamento dos Cursos de Graduação da Universidade Federal do Rio \nGrande do Norte – UFRN.  \n \n',
 'TÍTULO I  \nDAS DISPOSIÇÕES PRELIMINARES  \n \n',
 'Art. 2º  Este Regulamento tem por finalidade normatizar o ensino de graduação da Universidade \nFederal do Rio Grande do Norte - UFRN.  \n \nParágrafo único.   Os cursos que não possuem oferta regular serão regidos, conforme suas \nespecificidades, por este Regulame nto e por legislação específica.  \n \n',
 'TÍTULO II  \nDA GESTÃO DAS ATIVIDADES ACADÊMICAS  \n \n \n  \n \n',
 'Art. 3º  Na UFRN, a gestão das atividades acadêmicas, referentes à graduação, compete às \nCoordenações de Cursos, aos Departamentos Acadêmicos, às Unidades Acadêmicas Especial izadas, aos \nCentros Acadêmicos, à Pró -Reitoria de Graduação - PROGRAD e seus respectivos órgãos colegiados, e \naos docentes, cabendo a PROGRAD a sua coordenação geral.  \n \nParágrafo único.  As atividades a que se refere o caput  deste artigo são desenvolvid

In [133]:
import json
def create_dictionary(strings_list):
    document_dict = {}
    current_title = None
    current_section = None

    for string in strings_list:
        if string.startswith('TÍTULO'):
            current_title = string.strip().replace('\n', '')
            document_dict[current_title] = {}
            current_section = None  # Reinicia a seção atual ao encontrar um novo título
        elif string.startswith('Seção'):
            if current_title:
                current_section = string.strip().replace('\n', '')
                document_dict[current_title][current_section] = []
            else:
                # Seção sem um título correspondente - você pode decidir como lidar com isso.
                pass
        elif string.startswith('Art'):
            if current_title:
                if current_section:
                    document_dict[current_title][current_section].append(string.strip().replace('\n', ''))
                else:
                    # Seção não definida, adiciona diretamente ao título
                    document_dict[current_title][string.strip().replace('\n', '')] = None
            else:
                # Artigo sem título correspondente - você pode decidir como lidar com isso.
                document_dict[string.strip().replace('\n', '')] = None

    return document_dict

document_dict = create_dictionary(splits)
json_like = json.dumps(document_dict, indent=4, ensure_ascii=False)
print(json_like)


{
    "Art. 1º  Atualizar o Regulamento dos Cursos de Graduação da Universidade Federal do Rio Grande do Norte – UFRN.": null,
    "TÍTULO I  DAS DISPOSIÇÕES PRELIMINARES": {
        "Art. 2º  Este Regulamento tem por finalidade normatizar o ensino de graduação da Universidade Federal do Rio Grande do Norte - UFRN.   Parágrafo único.   Os cursos que não possuem oferta regular serão regidos, conforme suas especificidades, por este Regulame nto e por legislação específica.": null
    },
    "TÍTULO II  DA GESTÃO DAS ATIVIDADES ACADÊMICAS": {
        "Art. 3º  Na UFRN, a gestão das atividades acadêmicas, referentes à graduação, compete às Coordenações de Cursos, aos Departamentos Acadêmicos, às Unidades Acadêmicas Especial izadas, aos Centros Acadêmicos, à Pró -Reitoria de Graduação - PROGRAD e seus respectivos órgãos colegiados, e aos docentes, cabendo a PROGRAD a sua coordenação geral.   Parágrafo único.  As atividades a que se refere o caput  deste artigo são desenvolvidas n os prazos 

In [136]:
articles = []

for k, v in document_dict.items():
    if v is None:
        articles.append(k)
    elif isinstance(v, dict):
        for k2, v2 in v.items():
            if v2 is None:
                articles.append(k2)
            else:
                articles.append(v2)

articles

['Art. 1º  Atualizar o Regulamento dos Cursos de Graduação da Universidade Federal do Rio Grande do Norte – UFRN.',
 'Art. 2º  Este Regulamento tem por finalidade normatizar o ensino de graduação da Universidade Federal do Rio Grande do Norte - UFRN.   Parágrafo único.   Os cursos que não possuem oferta regular serão regidos, conforme suas especificidades, por este Regulame nto e por legislação específica.',
 'Art. 3º  Na UFRN, a gestão das atividades acadêmicas, referentes à graduação, compete às Coordenações de Cursos, aos Departamentos Acadêmicos, às Unidades Acadêmicas Especial izadas, aos Centros Acadêmicos, à Pró -Reitoria de Graduação - PROGRAD e seus respectivos órgãos colegiados, e aos docentes, cabendo a PROGRAD a sua coordenação geral.   Parágrafo único.  As atividades a que se refere o caput  deste artigo são desenvolvidas n os prazos estabelecidos no Calendário Universitário.',
 'Art. 4º   As rotinas administrativas serão processadas pelo sistema de gestão acadêmica.   Par